In [2]:
import pandas as pd
import os
import json

import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import GoogleV3
import reverse_geocode

import dask.dataframe as dd
import dask.bag as db
import json
import dask.array as da
import pyspark as spark
import shutil  

import dask.dataframe as dd
import dask.bag as db
import numpy as np

In [4]:
#renaming file names for better looping

month_list = ['january','february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
dict_month = dict(zip(['january','february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december'],['0'+str(i) if len(str(i)) == 1 else str(i) for i in list(range(1,13))]))

In [ ]:
for file in os.listdir('./Coronavirus (COVID-19) Geo-tagged Tweets Dataset/'):
    if file.startswith('orig'):
        for k,v in dict_month.items():
            if file.find(k) > 0:
                new_name = file
                new_name = new_name.replace(k, v+'_')
                os.rename('./Coronavirus (COVID-19) Geo-tagged Tweets Dataset/' + file, './Coronavirus (COVID-19) Geo-tagged Tweets Dataset/' + new_name)
                break

In [ ]:
#moving files to a separate folder

path = 'C:/Users/wedan/twitter germany/'
destination = 'C:/Users/wedan/twitter germany/Coronavirus (COVID-19) Geo-tagged Tweets Dataset/'

for f in os.listdir('./'):
    if any(x in f for x in month_list):
        try:
            dest = shutil.move(path + f.rstrip("\r\n"), destination + f.rstrip("\r\n"))
        except Exception as e:
            print(e, f)
            pass

In [42]:
def extract_ids(directory, f): 
    '''this function extracts tweet ids
    and sent scores from a csv file'''
    xtra = {}
    xtra1 = {}
    if f.endswith('csv'):
        data = pd.read_csv(directory + f, header=None)

        ids = data[0].tolist()
        sent = data.iloc[:,-1].tolist()
        ids = {'id': ids}
        sent = {'sent': sent}
    return ids, sent
    

df_ids = pd.DataFrame(columns = ['id'])
df_sents = pd.DataFrame(columns = ['sent'])

for f in os.listdir('./Coronavirus (COVID-19) Geo-tagged Tweets Dataset/'):
    df_ids = df_ids.append(pd.DataFrame(extract_ids('./Coronavirus (COVID-19) Geo-tagged Tweets Dataset/',f)[0]))
    df_sents = df_sents.append(pd.DataFrame(extract_ids('./Coronavirus (COVID-19) Geo-tagged Tweets Dataset/',f)[1]))

#df_ids.to_csv('ids_all_test.csv', header = False, index = False)
#df_sents.to_csv('sent_all_test.csv', header = False, index = False)

In [51]:
#splitting dataframe into multiple for better hydration

for idx,i in enumerate(np.array_split(df_ids, 5)):
    i.to_csv(str(idx), header = False, index = False)

In [50]:
def flatten(record):
    """this function chooses the necessary columns
    flattening the json file"""
    
    return {
        'created_at': record['created_at'],
        'full_text': record['full_text'],
        'location': record['user']['location'],
        'description': record['user']['description'],
        'geo': record['geo'],
        'coordinates': record['coordinates'],
        'full_name': record['place'],
        'lang': record['lang'],
    }

In [5]:
def flatten_dic(li, string):
    listi = []
    for x in li:
        try:
            listi.append(x[string])
        except TypeError:
            listi.append(None)
    return listi

In [6]:
big_df = []

for file in os.listdir('./'):
    if file.endswith('hydr'):
        print(file)
        df = pd.read_json(file, lines = True)
        df_filtered = df[['created_at', 'full_text', 'user', 'geo', 'place', 'lang']]
        place_list = df_filtered['place'].tolist()
        df_filtered = df_filtered.assign(place_type = flatten_dic(place_list, 'place_type'),full_name = flatten_dic(place_list, 'full_name'),country = flatten_dic(place_list, 'country'),loc_from_profile = flatten_dic(df_filtered['user'].tolist(), 'location'), coordinates = flatten_dic(df_filtered['geo'].tolist(), 'coordinates'))
        df_filtered.drop(['geo','place','user'], axis = 1, inplace = True)
        print(df_filtered)
        big_df.append(df_filtered)

0_hydr
                     created_at  \
0     2020-08-10 04:38:13+00:00   
1     2020-08-10 06:06:50+00:00   
2     2020-08-10 04:44:47+00:00   
3     2020-08-10 06:18:56+00:00   
4     2020-08-10 05:03:14+00:00   
...                         ...   
47744 2020-07-14 09:58:51+00:00   
47745 2020-07-14 09:52:29+00:00   
47746 2020-07-14 09:55:33+00:00   
47747 2020-07-14 10:02:33+00:00   
47748 2020-07-14 09:49:03+00:00   

                                               full_text lang place_type  \
0      Awesome BRAVE SkillAthletic class with Joe at ...   en       city   
1      So I love working from home, I don't need to w...   en       city   
2      Matching our swimsuits may have been a step to...   en       city   
3      1st staying overnight in a tent! Social distan...   en      admin   
4      Failed #FieldWork. Resident did not let us in ...   en       city   
...                                                  ...  ...        ...   
47744  Thank God for another day!!!\nBLA

In [7]:
big_df = pd.concat(big_df)

In [8]:
big_df

,created_at,full_text,lang,place_type,full_name,country,loc_from_profile,coordinates
0,2020-08-10 04:38:13+00:00,Awesome BRAVE SkillAthletic class with Joe at ...,en,city,"Newcastle, New South Wales",Australia,"Newcastle, NSW","[-32.88920957, 151.70629978]"
1,2020-08-10 06:06:50+00:00,"So I love working from home, I don't need to w...",en,city,"City of London, London",United Kingdom,"London, England","[51.5141, -0.10940000000000001]"
2,2020-08-10 04:44:47+00:00,Matching our swimsuits may have been a step to...,en,city,"Dallas, TX",United States,"Dallas, TX","[32.7774, -96.7977]"
3,2020-08-10 06:18:56+00:00,1st staying overnight in a tent! Social distan...,en,admin,"California, USA",United States,"San Francisco, CA","[37.41585485, -120.7100723]"
4,2020-08-10 05:03:14+00:00,Failed #FieldWork. Resident did not let us in ...,en,city,"Lungsod ng Caloocan, Pambansang Punong Rehiyon",Republika ng Pilipinas,"Oas, Philippines","[14.7513113, 121.07890606]"
...,...,...,...,...,...,...,...,...
47379,2020-09-10 04:19:19+00:00,Don’t judge me but during this #RonaDivide &am...,en,city,"Bowie, MD",United States,Bowie Maryland,"[38.9606, -76.7416]"
47380,2020-09-10 04:00:04+00:00,#RT #Trump Claims He Downplayed COVID-19 To Av...,en,city,"Warszawa, Polska",Polska,"Warszawa, Polska","[52.232222, 21.008333]"
47381,2020-09-10 04:12:32+00:00,Number of coronavirus cases in #Germany rises ...,en,city,"London, England",United Kingdom,"London, United Kingdom","[51.50538967, -0.12485951]"
47382,2020-09-10 04:18:10+00:00,"What lockdown? 🧔keep smiling guys, we can do t...",en,city,"Melbourne, Victoria",Australia,"Melbourne, VIC","[-37.8156, 144.9661]"


In [7]:
big_df.to_csv('filtered_data.csv')